In [1]:
%pylab

import math
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as sktr
import skimage as sk
import skimage.io as skio
from skimage.transform import rescale
import math
from skimage import filters
import scipy as sp
from skimage.draw import polygon
from scipy.interpolate import RectBivariateSpline, interp2d
from numpy.linalg import inv
from scipy.spatial import Delaunay
import os

def get_points(im1, im2,num_points=36):
    print('Please select 2 points in each image for alignment.')
    plt.imshow(im1)
    x = plt.ginput(num_points, timeout=200)
    plt.close()
    plt.imshow(im2)
    y = plt.ginput(num_points, timeout=200)
    plt.close()
    return (x,y)

def pts_from_name(im1name, im2name, num_points=36, imsize=(750,602,3)):
    im2 = skio.imread(im2name)/255
    im2 = sp.misc.imresize(im2, imsize)/255
    # low sf
    im = skio.imread(im1name)/255
    #   reshape
    im1 = sp.misc.imresize(im,imsize)/255
    pts_a,pts_b = get_points(im1, im2, num_points)
    #     add corners
    pts_a += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
    pts_b += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
    return pts_a, pts_b

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def avg_shape(a,b, t):
    return np.array([((1-t) * i[0] + t * j[0],(1-t) * i[1] + t * j[1])  for i,j in zip(a,b)])


In [3]:
# This does the right thing

def get_basis(m_tri):
#     m_Tri is an array of 3 lists, each with x,y coords, then also add extrabrow and column to account for translatiom
    b1 = m_tri[2] - m_tri[0]
    b2 = m_tri[1] - m_tri[0]
    part_b =  np.stack((b1,b2,m_tri[0]),axis=-1)
    add_on = np.array([0,0,1])
    return np.stack((part_b[0],part_b[1], add_on), axis=0)

In [4]:
# computer affine
# this computes the transformation from tri1 to tri2
def compute_affine(tri1, tri2):
    return get_basis(tri2).dot(np.linalg.inv(get_basis(tri1)))

In [5]:
def rng_tuple(z, z_max):
    rng = 5
    return (max(z-rng, 0) , min(z + rng, z_max - 1))    

def get_interp_vals(tg_tuple, src_im):
    x_b = math.floor(np.amin(tg_tuple[0]))
    x_e = math.floor(np.amax(tg_tuple[0]))
    y_b = math.floor(np.amin(tg_tuple[1]))
    y_e = math.floor(np.amax(tg_tuple[1]))
    xold = np.arange(x_b, x_e)
    yold = np.arange(y_b, y_e)
    if (abs(x_b - x_e) < 4):
        return src_im[tg_tuple[1], tg_tuple[0]]
    if (abs(y_b - y_e) < 4):
        return src_im[tg_tuple[1], tg_tuple[0]]
#     print('wut')
    src_im_inter = RectBivariateSpline(yold, xold, src_im[y_b: y_e,x_b: x_e], ky=2)
    f = src_im_inter(tg_tuple[1], tg_tuple[0],grid=False)
    return f  


def merge_triangles(srcTriList, tgtTriList,src_im, frac = 0.5):
    new_im = np.zeros(np.shape(src_im))
    src_im_disp = np.dstack((src_im,src_im,src_im))
    for stri, ttri in zip(srcTriList, tgtTriList):
        height = len(src_im)
        width = len(src_im[0])
        stform = compute_affine(stri, ttri)
        r = [s[0] for s in ttri]
        c = [s[1] for s in ttri]
        rr, cc = polygon(r,c, shape=(width,height))
        new_coords = inv(stform).dot(np.array([rr,cc,np.ones(np.shape(rr))])).astype(int)
        new_im[cc,rr] = np.reshape(get_interp_vals(new_coords,src_im), np.shape(new_im[cc,rr]))
    return frac * new_im 
            

        

In [6]:
def sc_disp(im_c, cover_x=[0], cover_y=[0]):
    im_c[cover_y, cover_x] = 1
    plt.imshow(np.dstack((im_c,im_c,im_c)))
    plt.show()

In [7]:
im2 = skio.imread('george_small.jpg')/255
im1 = skio.imread('eric_small.jpg')/255
im1 = sp.misc.imresize(im1,im2.shape )/255

x2 = [(54.37187119353564, 187.56292782514709), (114.84640326648253, 78.194093225136839), (222.92854569472797, 22.866329839249261), (382.47837499356655, 34.446559385132673), (458.39321312769135, 106.50132100396297), (491.84720959357679, 192.70969651220639), (135.43347801471975, 639.19188011460142), (72.385561598243214, 545.26335157576898), (58.23194770883012, 423.02759525811041), (71.098869426478387, 289.21160939456843), (119.99317195354183, 175.98269827926367), (219.06846917943352, 129.66178009572991), (360.60460807356452, 136.09524095455401), (446.81298358180783, 209.43669474514911), (476.40690353239887, 321.37891368868907), (490.56051742181199, 448.76143869340694), (454.53313661239685, 561.9903498087117), (379.90499065003684, 644.33864880166061), (249.94908130178936, 717.68010259225582), (85.25248331589151, 312.37206848633525), (145.72701538883837, 277.63137984868501), (229.36200655355213, 299.50514676868704), (347.7376863559162, 307.22529979927594), (395.34529671121476, 287.92491722280363), (463.53998181475066, 321.37891368868907), (171.4608588241349, 352.25952581104491), (376.04491413474244, 361.26637101339873), (202.34147094649074, 473.20858995693868), (278.25630908061555, 356.11960232633942), (341.30422549709209, 480.92874298752764), (280.8296934241452, 455.19489955223111), (184.32778054178317, 576.14396369812482), (269.24946387826174, 551.69681243459308), (350.31107069944591, 587.72419324400835), (310.42361337473619, 605.73788364871587), (237.0821595841411, 607.02457582048066)]
y2 = [(65.401792072157605, 180.16528771489618), (111.17846970262113, 60.637296124241402), (202.73182496354818, 9.7743209792819243), (379.48066359228233, 12.317469736529915), (455.77512630972149, 88.611932453969075), (535.88431216303275, 204.32520090875187), (190.01608117730834, 657.00567969889119), (123.89421348886103, 567.99547319521207), (109.90689532399713, 480.25684107015701), (97.191151537757293, 334.0257875283985), (137.88153165372486, 154.73380014241638), (239.60748194364382, 101.32767624020892), (379.48066359228233, 100.05610186158492), (465.94772133871345, 159.82009765691237), (493.92235766844112, 328.93949001390263), (493.92235766844112, 490.42943609914892), (473.57716761045731, 573.08177070970805), (415.08474619375397, 643.01836153402735), (296.82832898172319, 684.98031602861886), (121.35106473161304, 331.4826387711505), (167.12774236207653, 275.53336611169516), (259.95267200162755, 295.87855616967897), (359.13547353429851, 297.15013054830297), (429.07206435861781, 272.99021735444717), (468.49087009596144, 322.58161812078265), (200.18867620630019, 328.93949001390263), (398.55427927164214, 326.39634125665464), (243.42220507951575, 444.65275846868536), (298.09990336034718, 322.58161812078265), (347.69130412668267, 447.19590722593335), (294.2851802244752, 454.82535349767727), (210.36127123529209, 546.37870875860426), (289.19888270997933, 533.66296497236453), (379.48066359228233, 541.29241124410839), (346.41972974805867, 564.1807500593402), (287.92730833135533, 574.35334508833205)]

x2+= [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]

y2+= [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]


point5 = avg_shape(x2,y2, 0.5)
tri = Delaunay(point5)
t_simp = tri.simplices
triPoints = point5[t_simp]
xPoints = np.array(x2)[t_simp]
yPoints = np.array(y2)[t_simp]

# get the middle image for me and george clooney


[im1_r, im1_g, im1_b] = np.dsplit(im1, 3)
[im2_r, im2_g, im2_b] = np.dsplit(im2, 3)


newIm1_r  = merge_triangles(xPoints, triPoints, im1_r.astype(np.float32))
newIm1_g  = merge_triangles(xPoints, triPoints, im1_g.astype(np.float32))
newIm1_b  = merge_triangles(xPoints, triPoints, im1_b.astype(np.float32))


newIm2_r= merge_triangles(yPoints, triPoints, im2_r.astype(np.float32))
newIm2_g= merge_triangles(yPoints, triPoints, im2_g.astype(np.float32))
newIm2_b= merge_triangles(yPoints, triPoints, im2_b.astype(np.float32))





newIm3_r = newIm1_r + newIm2_r
newIm3_g = newIm1_g + newIm2_g
newIm3_b = newIm1_b + newIm2_b
im1_n = np.dstack((newIm1_r, newIm1_g, newIm1_b)) 
im2_n = np.dstack((newIm2_r, newIm2_g, newIm2_b)) 
im3 = np.dstack((newIm3_r, newIm3_g, newIm3_b)) 




# first index is length, second is width
# len(im2) is length and len(im2[0]) is width

plt.imshow(im3)
plt.show()

fname = 'eric_george_halfway.jpg'
skio.imsave(fname, im3)


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
# sc_disp(im1_r)

In [9]:
# The idea is that we input two images. If we do not already have triangulation points,
# Then we plot our own. It might be better to keep plot function and the face function seperate, not sure. 
# Else, we would find triangulation,find avg triangles. 
# Then loop. 
# One function
# get x and y and transition stage triangles with simplices. 
# Ask how this works, do we always copy from two source images. 
# Merge faces 6 times, 3 for each image's channels. 
# Save each image. 

In [10]:
def morph(im1, im2, im1_pts, im2_pts, tri_simp, warp_frac, dissolve_frac, caricature=None):
    im1_tris = np.array(im1_pts)[tri_simp]
    im2_tris = np.array(im2_pts)[tri_simp] 
    im1_ch = np.dsplit(im1, im1.shape[-1])
    im2_ch = np.dsplit(im2, im2.shape[-1])
#     merging 
    if (caricature):
        warp_pts = avg_shape(im1_pts,im2_pts, -warp_frac * caricature)
        warp_tris = warp_pts[tri_simp]
        newIm1_ch = [merge_triangles(im1_tris, warp_tris, i.astype(np.float32), (1 + dissolve_frac * caricature)) for i in im1_ch]
        newIm2_ch = [merge_triangles(im2_tris, warp_tris, i.astype(np.float32), (-dissolve_frac*caricature)) for i in im2_ch]
    else:
        warp_pts = avg_shape(im1_pts,im2_pts, warp_frac)
        warp_tris = warp_pts[tri_simp]
        newIm1_ch = [merge_triangles(im1_tris, warp_tris, i.astype(np.float32), (1 - dissolve_frac)) for i in im1_ch]
        newIm2_ch = [merge_triangles(im2_tris, warp_tris, i.astype(np.float32), (dissolve_frac)) for i in im2_ch]
    im1_n = np.dstack(tuple(newIm1_ch))
    im2_n = np.dstack(tuple(newIm2_ch))

    morph_im = np.dstack(tuple([a + b for a,b in zip(newIm1_ch, newIm2_ch)])) 
    return morph_im

In [11]:
def give_tri(pts_a, pts_b):
    point5 = avg_shape(pts_a,pts_b, 0.5)
    return Delaunay(point5)

In [12]:
def c_lbl(caricature):
    if (caricature == None):
        return ''
    return 'caricature' + str(caricature)


def create_morph(imname_a, imname_b, pts_a=None, pts_b=None, num_points = 36, resize=(750,602,3),num_frames=45,caricature=None):
    # high sf
    im2 = skio.imread(imname_b)/255
    im2 = sp.misc.imresize(im2, resize)/255
    # low sf
    im = skio.imread(imname_a)/255
#     reshape
    im1 = sp.misc.imresize(im, resize)/255
#     print(np.shape(im1))

    
#     give points if not given already
    if ((pts_a == None) or (pts_b == None)):
        pts_a,pts_b = get_points(im1, im2, num_points)
#     add corners
        pts_a += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
        pts_b += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
    tri = give_tri(pts_a, pts_b)
    tri_simp = tri.simplices
#     we include 46 so we can have the number 45
# create 45 frames
#     save first image
    fname = 'merges/' + imname_a[:-4] + '_' + imname_b[:-4] + '_' + c_lbl(caricature) + '_' + str(1000) + imname_b[-4:len(imname_b)]
    skio.imsave(fname, im1)
    for i in range(1,num_frames):
#         get the morphed image and save it somewhere. 
        increment = 1 / num_frames

        new_im = morph(im1, im2, pts_a, pts_b, tri_simp, i * increment, i * increment, caricature)
        fname = 'merges/' + imname_a[:-4] + '_' + imname_b[:-4] + '_' + c_lbl(caricature) + '_' +  str(1000 + i) + imname_b[-4:len(imname_b)]
        skio.imsave(fname, np.clip(new_im,0.0,1.0))
#     save last image
    if (caricature == None):
        fname = 'merges/' + imname_a[:-4] + '_' + imname_b[:-4] +  '_' + c_lbl(caricature) + '_' + str(1000 + num_frames) + imname_b[-4:len(imname_b)]
        skio.imsave(fname, im2)
    all_names = 'merges/' + imname_a[:-4] + '_' + imname_b[:-4] + '_' + c_lbl(caricature) + '_' + '*' + imname_b[-4:len(imname_b)]
    o_name = 'gifs/' + imname_a[:-4] + '_' + imname_b[:-4] + '_' + c_lbl(caricature) + '_' + 'merge.gif'
    os.system('convert -delay 10 {0} {1}'.format(all_names, o_name))
    return pts_a, pts_b


In [13]:
im2 = skio.imread('george_small.jpg')/255
x2 = [(54.37187119353564, 187.56292782514709), (114.84640326648253, 78.194093225136839), (222.92854569472797, 22.866329839249261), (382.47837499356655, 34.446559385132673), (458.39321312769135, 106.50132100396297), (491.84720959357679, 192.70969651220639), (135.43347801471975, 639.19188011460142), (72.385561598243214, 545.26335157576898), (58.23194770883012, 423.02759525811041), (71.098869426478387, 289.21160939456843), (119.99317195354183, 175.98269827926367), (219.06846917943352, 129.66178009572991), (360.60460807356452, 136.09524095455401), (446.81298358180783, 209.43669474514911), (476.40690353239887, 321.37891368868907), (490.56051742181199, 448.76143869340694), (454.53313661239685, 561.9903498087117), (379.90499065003684, 644.33864880166061), (249.94908130178936, 717.68010259225582), (85.25248331589151, 312.37206848633525), (145.72701538883837, 277.63137984868501), (229.36200655355213, 299.50514676868704), (347.7376863559162, 307.22529979927594), (395.34529671121476, 287.92491722280363), (463.53998181475066, 321.37891368868907), (171.4608588241349, 352.25952581104491), (376.04491413474244, 361.26637101339873), (202.34147094649074, 473.20858995693868), (278.25630908061555, 356.11960232633942), (341.30422549709209, 480.92874298752764), (280.8296934241452, 455.19489955223111), (184.32778054178317, 576.14396369812482), (269.24946387826174, 551.69681243459308), (350.31107069944591, 587.72419324400835), (310.42361337473619, 605.73788364871587), (237.0821595841411, 607.02457582048066)]
y2 = [(65.401792072157605, 180.16528771489618), (111.17846970262113, 60.637296124241402), (202.73182496354818, 9.7743209792819243), (379.48066359228233, 12.317469736529915), (455.77512630972149, 88.611932453969075), (535.88431216303275, 204.32520090875187), (190.01608117730834, 657.00567969889119), (123.89421348886103, 567.99547319521207), (109.90689532399713, 480.25684107015701), (97.191151537757293, 334.0257875283985), (137.88153165372486, 154.73380014241638), (239.60748194364382, 101.32767624020892), (379.48066359228233, 100.05610186158492), (465.94772133871345, 159.82009765691237), (493.92235766844112, 328.93949001390263), (493.92235766844112, 490.42943609914892), (473.57716761045731, 573.08177070970805), (415.08474619375397, 643.01836153402735), (296.82832898172319, 684.98031602861886), (121.35106473161304, 331.4826387711505), (167.12774236207653, 275.53336611169516), (259.95267200162755, 295.87855616967897), (359.13547353429851, 297.15013054830297), (429.07206435861781, 272.99021735444717), (468.49087009596144, 322.58161812078265), (200.18867620630019, 328.93949001390263), (398.55427927164214, 326.39634125665464), (243.42220507951575, 444.65275846868536), (298.09990336034718, 322.58161812078265), (347.69130412668267, 447.19590722593335), (294.2851802244752, 454.82535349767727), (210.36127123529209, 546.37870875860426), (289.19888270997933, 533.66296497236453), (379.48066359228233, 541.29241124410839), (346.41972974805867, 564.1807500593402), (287.92730833135533, 574.35334508833205)]
x2+= [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
y2+= [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]

In [14]:

create_morph('eric_small.jpg', 'george_small.jpg', x2, y2)

([(54.37187119353564, 187.56292782514709),
  (114.84640326648253, 78.19409322513684),
  (222.92854569472797, 22.86632983924926),
  (382.47837499356655, 34.44655938513267),
  (458.39321312769135, 106.50132100396297),
  (491.8472095935768, 192.7096965122064),
  (135.43347801471975, 639.1918801146014),
  (72.38556159824321, 545.263351575769),
  (58.23194770883012, 423.0275952581104),
  (71.09886942647839, 289.2116093945684),
  (119.99317195354183, 175.98269827926367),
  (219.06846917943352, 129.6617800957299),
  (360.6046080735645, 136.09524095455401),
  (446.8129835818078, 209.4366947451491),
  (476.40690353239887, 321.37891368868907),
  (490.560517421812, 448.76143869340694),
  (454.53313661239685, 561.9903498087117),
  (379.90499065003684, 644.3386488016606),
  (249.94908130178936, 717.6801025922558),
  (85.25248331589151, 312.37206848633525),
  (145.72701538883837, 277.631379848685),
  (229.36200655355213, 299.50514676868704),
  (347.7376863559162, 307.22529979927594),
  (395.34529671

In [15]:
# # #####################toggle pts_a,pts_b,pts_c, pts_d, pts_e, pts_f
# #####################this might not work because of some sort of np estimation thing, or might give a missing triangle
# # That is why I did the below and added the corners later
# # These points will tell you which ones I chose in case you want to do it manually
x1 = [(83.37337662337643, 296.57792207792193), (170.62987012986997, 268.16883116883105), (229.47727272727258, 296.57792207792193), (162.51298701298691, 296.57792207792193), (330.93831168831156, 302.66558441558436), (403.99025974025966, 282.37337662337654), (472.98376623376606, 308.75324675324669), (406.0194805194804, 310.78246753246742), (111.78246753246736, 351.366883116883), (172.65909090909071, 339.1915584415583), (213.24350649350635, 359.48376623376612), (156.42532467532448, 369.62987012987003), (341.08441558441547, 373.68831168831161), (401.96103896103881, 343.24999999999989), (452.69155844155836, 355.42532467532459), (393.84415584415575, 373.68831168831161), (213.24350649350635, 465.00324675324669), (284.2662337662336, 347.30844155844147), (341.08441558441547, 469.06168831168821), (276.14935064935054, 495.44155844155836), (276.14935064935054, 458.91558441558431), (184.83441558441547, 574.58116883116872), (272.09090909090895, 548.20129870129858), (347.17207792207779, 578.63961038961031), (266.00324675324663, 605.01948051948045), (71.198051948051784, 337.16233766233756), (69.168831168830991, 548.20129870129858), (253.82792207792198, 706.48051948051943), (460.80844155844142, 586.75649350649337), (499.3636363636362, 385.86363636363626)]

y1 = [(121.92857142857127, 320.92857142857133), (176.71753246753229, 262.08116883116872), (253.82792207792198, 298.60714285714278), (170.62987012986997, 302.66558441558436), (355.28896103896096, 296.57792207792193), (422.25324675324663, 262.08116883116872), (464.866883116883, 298.60714285714278), (408.04870129870113, 300.63636363636351), (154.39610389610374, 343.24999999999989), (203.09740259740244, 316.87012987012974), (266.00324675324663, 324.98701298701292), (186.8636363636362, 355.42532467532459), (345.14285714285705, 337.16233766233756), (406.0194805194804, 314.84090909090901), (460.80844155844142, 343.24999999999989), (379.63961038961025, 347.30844155844147), (231.50649350649331, 448.76948051948045), (296.44155844155836, 310.78246753246742), (347.17207792207779, 444.71103896103887), (296.44155844155836, 467.03246753246742), (296.44155844155836, 438.62337662337654), (217.30194805194793, 540.08441558441552), (284.2662337662336, 529.93831168831161), (379.63961038961025, 538.05519480519479), (300.49999999999983, 564.43506493506493), (97.577922077921926, 343.24999999999989), (123.95779220779207, 558.34740259740261), (282.23701298701286, 690.2467532467532), (481.10064935064923, 564.43506493506493), (507.48051948051938, 333.10389610389598)]

x2 = [(54.37187119353564, 187.56292782514709), (114.84640326648253, 78.194093225136839), (222.92854569472797, 22.866329839249261), (382.47837499356655, 34.446559385132673), (458.39321312769135, 106.50132100396297), (491.84720959357679, 192.70969651220639), (135.43347801471975, 639.19188011460142), (72.385561598243214, 545.26335157576898), (58.23194770883012, 423.02759525811041), (71.098869426478387, 289.21160939456843), (119.99317195354183, 175.98269827926367), (219.06846917943352, 129.66178009572991), (360.60460807356452, 136.09524095455401), (446.81298358180783, 209.43669474514911), (476.40690353239887, 321.37891368868907), (490.56051742181199, 448.76143869340694), (454.53313661239685, 561.9903498087117), (379.90499065003684, 644.33864880166061), (249.94908130178936, 717.68010259225582), (85.25248331589151, 312.37206848633525), (145.72701538883837, 277.63137984868501), (229.36200655355213, 299.50514676868704), (347.7376863559162, 307.22529979927594), (395.34529671121476, 287.92491722280363), (463.53998181475066, 321.37891368868907), (171.4608588241349, 352.25952581104491), (376.04491413474244, 361.26637101339873), (202.34147094649074, 473.20858995693868), (278.25630908061555, 356.11960232633942), (341.30422549709209, 480.92874298752764), (280.8296934241452, 455.19489955223111), (184.32778054178317, 576.14396369812482), (269.24946387826174, 551.69681243459308), (350.31107069944591, 587.72419324400835), (310.42361337473619, 605.73788364871587), (237.0821595841411, 607.02457582048066)]

y2 = [(65.401792072157605, 180.16528771489618), (111.17846970262113, 60.637296124241402), (202.73182496354818, 9.7743209792819243), (379.48066359228233, 12.317469736529915), (455.77512630972149, 88.611932453969075), (535.88431216303275, 204.32520090875187), (190.01608117730834, 657.00567969889119), (123.89421348886103, 567.99547319521207), (109.90689532399713, 480.25684107015701), (97.191151537757293, 334.0257875283985), (137.88153165372486, 154.73380014241638), (239.60748194364382, 101.32767624020892), (379.48066359228233, 100.05610186158492), (465.94772133871345, 159.82009765691237), (493.92235766844112, 328.93949001390263), (493.92235766844112, 490.42943609914892), (473.57716761045731, 573.08177070970805), (415.08474619375397, 643.01836153402735), (296.82832898172319, 684.98031602861886), (121.35106473161304, 331.4826387711505), (167.12774236207653, 275.53336611169516), (259.95267200162755, 295.87855616967897), (359.13547353429851, 297.15013054830297), (429.07206435861781, 272.99021735444717), (468.49087009596144, 322.58161812078265), (200.18867620630019, 328.93949001390263), (398.55427927164214, 326.39634125665464), (243.42220507951575, 444.65275846868536), (298.09990336034718, 322.58161812078265), (347.69130412668267, 447.19590722593335), (294.2851802244752, 454.82535349767727), (210.36127123529209, 546.37870875860426), (289.19888270997933, 533.66296497236453), (379.48066359228233, 541.29241124410839), (346.41972974805867, 564.1807500593402), (287.92730833135533, 574.35334508833205)]

pts_a=[(61.051948051947875, 199.17532467532453), (101.63636363636346, 93.655844155844079), (223.38961038961025, 22.633116883116713), (428.34090909090895, 24.662337662337563), (485.15909090909082, 87.568181818181642), (537.91883116883105, 197.1461038961038), (87.431818181818016, 331.07467532467524), (115.84090909090895, 156.56168831168816), (237.59415584415575, 103.80194805194787), (401.96103896103881, 107.86038961038946), (491.24675324675314, 172.79545454545439), (521.68506493506493, 322.95779220779207), (499.3636363636362, 440.65259740259734), (477.04220779220765, 489.35389610389603), (389.78571428571416, 556.31818181818176), (306.58766233766215, 562.40584415584408), (219.33116883116867, 554.28896103896091), (152.366883116883, 507.61688311688306), (109.75324675324657, 460.9448051948051), (115.84090909090895, 241.78896103896091), (182.80519480519462, 209.32142857142844), (253.82792207792198, 235.70129870129858), (379.63961038961025, 237.73051948051932), (440.5162337662336, 215.40909090909076), (501.39285714285705, 251.93506493506482), (205.12662337662317, 294.54870129870119), (418.19480519480504, 294.54870129870119), (239.62337662337649, 398.03896103896096), (304.55844155844142, 276.28571428571422), (381.66883116883099, 396.00974025974017), (310.64610389610374, 365.5714285714285), (237.59415584415575, 460.9448051948051), (312.67532467532459, 438.62337662337654), (401.96103896103881, 467.03246753246742), (355.28896103896096, 487.32467532467524), (278.17857142857127, 489.35389610389603), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]



pts_b = [(91.490259740259603, 152.50324675324669), (166.57142857142838, 46.983766233766119), (266.00324675324663, 26.6915584415583), (406.0194805194804, 36.837662337662209), (493.27597402597388, 83.50974025974017), (525.7435064935064, 154.53246753246742), (119.89935064935048, 341.22077922077915), (134.10389610389592, 223.52597402597394), (192.95129870129853, 134.2402597402596), (424.28246753246736, 120.03571428571411), (493.27597402597388, 211.35064935064929), (495.30519480519473, 333.10389610389598), (493.27597402597388, 422.38961038961031), (450.66233766233751, 483.26623376623371), (381.66883116883099, 533.9967532467532), (306.58766233766215, 550.23051948051943), (237.59415584415575, 542.11363636363626), (162.51298701298691, 497.47077922077915), (144.24999999999983, 465.00324675324669), (170.62987012986997, 227.58441558441552), (203.09740259740244, 205.26298701298686), (241.65259740259722, 213.37987012987003), (367.46428571428561, 213.37987012987003), (420.22402597402589, 193.08766233766221), (466.89610389610374, 241.78896103896091), (219.33116883116867, 270.19805194805178), (397.90259740259722, 270.19805194805178), (241.65259740259722, 347.30844155844147), (304.55844155844142, 247.87662337662323), (363.40584415584402, 353.3961038961038), (292.38311688311677, 314.84090909090901), (217.30194805194793, 436.59415584415575), (296.44155844155836, 389.92207792207785), (383.69805194805184, 432.53571428571422), (330.93831168831156, 450.79870129870119), (266.00324675324663, 460.9448051948051), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

pts_c = [(61.051948051947875, 195.11688311688295), (146.27922077922062, 69.305194805194674), (481.10064935064923, 57.129870129870028), (537.91883116883105, 205.26298701298686), (477.04220779220765, 489.35389610389603), (314.70454545454533, 568.4935064935064), (146.27922077922062, 501.52922077922068), (140.1915584415583, 296.57792207792193), (203.09740259740244, 276.28571428571422), (253.82792207792198, 296.57792207792193), (201.0681818181817, 306.72402597402584), (373.55194805194793, 296.57792207792193), (428.34090909090895, 282.37337662337654), (487.18831168831156, 300.63636363636351), (424.28246753246736, 312.81168831168816), (241.65259740259722, 391.95129870129858), (312.67532467532459, 292.51948051948045), (375.58116883116867, 391.95129870129858), (310.64610389610374, 375.71753246753235), (233.53571428571416, 465.00324675324669), (312.67532467532459, 432.53571428571422), (397.90259740259722, 465.00324675324669), (314.70454545454533, 487.32467532467524), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

pts_d = [(89.46103896103881, 144.38636363636351), (178.74675324675314, 44.954545454545269), (466.89610389610374, 61.188311688311501), (525.7435064935064, 162.64935064935048), (462.83766233766227, 483.26623376623371), (314.70454545454533, 548.20129870129858), (170.62987012986997, 503.55844155844147), (174.68831168831156, 274.25649350649337), (219.33116883116867, 262.08116883116872), (261.94480519480504, 276.28571428571422), (219.33116883116867, 284.40259740259728), (357.3181818181817, 274.25649350649337), (395.87337662337649, 264.11038961038946), (448.63311688311677, 274.25649350649337), (391.8149350649349, 282.37337662337654), (237.59415584415575, 359.48376623376612), (300.49999999999983, 268.16883116883105), (361.37662337662329, 353.3961038961038), (292.38311688311677, 322.95779220779207), (215.27272727272708, 438.62337662337654), (302.52922077922068, 389.92207792207785), (387.75649350649343, 428.47727272727263), (296.44155844155836, 454.85714285714278), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

x4 = [(365.43506493506476, 144.38636363636351), (182.80519480519462, 156.56168831168816), (136.13311688311671, 320.92857142857133), (399.93181818181807, 324.98701298701292), (320.79220779220765, 324.98701298701292), (397.90259740259722, 389.92207792207785), (403.99025974025966, 452.82792207792198), (328.90909090909082, 454.85714285714278), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

y4=[(395.87337662337649, 113.94805194805178), (227.44805194805184, 118.00649350649337), (140.1915584415583, 324.98701298701292), (436.45779220779212, 347.30844155844147), (363.40584415584402, 343.24999999999989), (428.34090909090895, 418.33116883116872), (424.28246753246736, 473.1201298701298), (361.37662337662329, 475.14935064935059), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

pts_e=[(59.022727272727082, 176.85389610389598), (115.84090909090895, 65.246753246753087), (221.36038961038952, 14.516233766233654), (369.49350649350635, 20.603896103895977), (452.69155844155836, 81.48051948051932), (493.27597402597388, 199.17532467532453), (65.110389610389461, 324.98701298701292), (125.9870129870128, 172.79545454545439), (203.09740259740244, 140.32792207792193), (369.49350649350635, 150.47402597402584), (450.66233766233751, 209.32142857142844), (487.18831168831156, 337.16233766233756), (491.24675324675314, 454.85714285714278), (452.69155844155836, 574.58116883116872), (347.17207792207779, 680.10064935064929), (253.82792207792198, 710.53896103896102), (158.45454545454533, 667.92532467532465), (77.285714285714107, 570.52272727272725), (61.051948051947875, 422.38961038961031), (93.519480519480339, 292.51948051948045), (154.39610389610374, 280.34415584415569), (227.44805194805184, 300.63636363636351), (341.08441558441547, 304.6948051948051), (410.07792207792198, 288.46103896103887), (466.89610389610374, 318.89935064935059), (172.65909090909071, 355.42532467532459), (377.61038961038952, 361.51298701298691), (209.18506493506476, 465.00324675324669), (282.23701298701286, 351.366883116883), (341.08441558441547, 485.29545454545445), (278.17857142857127, 462.97402597402589), (172.65909090909071, 576.61038961038957), (270.06168831168821, 552.25974025974017), (343.1136363636362, 584.72727272727263), (304.55844155844142, 611.10714285714278), (231.50649350649331, 605.01948051948045), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

pts_f=[(63.081168831168668, 170.76623376623365), (119.89935064935048, 87.568181818181642), (211.21428571428561, 34.808441558441473), (369.49350649350635, 38.866883116882946), (477.04220779220765, 95.685064935064815), (535.8896103896102, 178.88311688311671), (75.256493506493314, 377.74675324675314), (97.577922077921926, 211.35064935064929), (188.89285714285694, 152.50324675324669), (395.87337662337649, 152.50324675324669), (501.39285714285705, 231.642857142857), (523.71428571428555, 383.83441558441547), (517.62662337662323, 503.55844155844147), (491.24675324675314, 594.87337662337654), (416.16558441558431, 663.86688311688306), (302.52922077922068, 692.27597402597394), (215.27272727272708, 674.01298701298697), (115.84090909090895, 607.04870129870119), (89.46103896103881, 521.82142857142844), (125.9870129870128, 347.30844155844147), (172.65909090909071, 306.72402597402584), (241.65259740259722, 329.04545454545439), (361.37662337662329, 322.95779220779207), (422.25324675324663, 310.78246753246742), (481.10064935064923, 341.22077922077915), (197.00974025974011, 371.65909090909082), (408.04870129870113, 367.60064935064923), (249.7694805194804, 473.1201298701298), (302.52922077922068, 385.86363636363626), (355.28896103896096, 483.26623376623371), (308.616883116883, 483.26623376623371), (213.24350649350635, 576.61038961038957), (306.58766233766215, 546.17207792207785), (397.90259740259722, 570.52272727272725), (347.17207792207779, 588.78571428571422), (276.14935064935054, 592.84415584415569), (0.0, 0.0), (602.0, 0), (602.0, 750.0), (0.0, 750.0)]

In [16]:
# im2 = skio.imread('daniel_lin.jpg')/255
# im2 = sp.misc.imresize(im2, (750,602,3))/255
# # low sf
# im = skio.imread('daniel_liu.jpg')/255
# #   reshape
# im1 = sp.misc.imresize(im,(750,602,3) )/255
# pts_a,pts_b = get_points(im1, im2, 36)
# #     add corners
# pts_a += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
# pts_b += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]



In [17]:
# pts_c,pts_d = get_points(im1, im2, 23)
# #     add corners
# pts_c += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]
# pts_d += [(0.0,0.0), (float(len(im2[0])),0), (float(len(im2[0])), float(len(im2))),(0.0, float(len(im2)))]


In [18]:
x3, y3 = create_morph('daniel_liu.jpg', 'daniel_lin.jpg',pts_c, pts_d, 36)

In [19]:
# so do both daniels and then move on to the median faces

In [21]:
x4, y4 = create_morph('alexander_hamilton.png', 'thomas_jefferson.png',x4,y4,num_points=8)

In [22]:
# take parser, find avg shape, then do merge triangles and print, then merge everything together
# write the parser here, task one
def asf_parser(asf_name):
#     takes in asf file name, spits out list of coords
    coord_lst = []
    with open(asf_name, 'r') as f:
        for f_line in f:
            f_lst = f_line.split()
            if (len(f_lst) > 0):
                if (f_lst[0].isdigit() and int(f_lst[0]) in range(0,7)):
                    coord_lst.append((float(f_lst[2]), float(f_lst[3])))
    return coord_lst

In [23]:
asf_parser('imm_face_db/01-1m.asf')

[(0.35763609, 0.64077979),
 (0.36267641, 0.68243247),
 (0.3747732, 0.72945964),
 (0.39896673, 0.78051776),
 (0.42215222, 0.81679589),
 (0.4594506, 0.84769946),
 (0.4907006, 0.85441762),
 (0.53707159, 0.83963764),
 (0.56428933, 0.81007773),
 (0.58243448, 0.77245593),
 (0.59755546, 0.72811604),
 (0.60763609, 0.6904943),
 (0.61469257, 0.65018529),
 (0.5844506, 0.50776005),
 (0.56932962, 0.49298009),
 (0.5532006, 0.48760554),
 (0.5310232, 0.49029282),
 (0.5229587, 0.50641644),
 (0.54110384, 0.51179093),
 (0.55622482, 0.51582187),
 (0.57235384, 0.51582187),
 (0.39291835, 0.50507277),
 (0.41106352, 0.48894918),
 (0.43021673, 0.48088738),
 (0.4483619, 0.49163646),
 (0.46449092, 0.50507277),
 (0.44634578, 0.51044732),
 (0.4282006, 0.51179093),
 (0.41005543, 0.50910372),
 (0.50783771, 0.47551286),
 (0.52094257, 0.44864017),
 (0.57336187, 0.4378911),
 (0.60561997, 0.45670196),
 (0.61166835, 0.50238556),
 (0.47658771, 0.47282559),
 (0.45743448, 0.44460928),
 (0.40501511, 0.43386021),
 (0.37578124

In [24]:
def m_or_f(a):
    if os.path.exists(a):
        return a
    else:
        return a[:-5] + 'f.asf'
    

In [25]:
asf_lst = [m_or_f('imm_face_db/0{0}-1m.asf'.format(str(i))) if i < 10 else m_or_f('imm_face_db/{0}-1m.asf'.format(str(i))) for i in range(1,41)]

In [26]:
coord_lst = [asf_parser(a) for a in asf_lst]

In [27]:
coord_array = np.array(coord_lst)

In [28]:
coord_mean = np.mean(coord_array, axis=0)

In [29]:
# now that I got coords, try to merge all faces. 
# get the triangles first
mean_tri = Delaunay(coord_mean)
mean_simp = mean_tri.simplices

In [30]:
def make_morph_lst(im_lst, pts_lst, mean_pts):
    
    if (len(im_lst) != len(pts_lst)):
        return "error"
    
#     multiple length and also add the edges
    pts_lst = [[(im.shape[1] * coord[0], im.shape[0] * coord[1]) for coord in coords] for im, coords in zip(im_lst, pts_lst) ]
    first_im = im_lst[0]
    mean_pts = [(first_im.shape[1] * coord[0], first_im.shape[0] * coord[1]) for coord in mean_pts]
    for i in range(0,len(pts_lst)):
        pts_lst[i] += [(0.0,0.0), (float(len(first_im[0])),0), (float(len(first_im[0])), float(len(first_im))),(0.0, float(len(first_im)))]
    mean_pts += [(0.0,0.0), (float(len(first_im[0])),0), (float(len(first_im[0])), float(len(first_im))),(0.0, float(len(first_im)))]
    morph_lst = []
    tri = Delaunay(mean_pts)
    tri_simp = tri.simplices
    mean_tris = np.array(mean_pts)[tri_simp]
    dissolve_frac = 1
    points = np.array(mean_pts)
    plt.triplot(points[:,0], points[:,1], tri.simplices.copy())
    plt.plot(points[:,0], points[:,1], 'o')
    plt.show()
    for im,pts in zip(im_lst, pts_lst):
        im_tris = np.array(pts)[tri_simp]
        im_ch = np.dsplit(im, im.shape[-1])
#     merging 
        newIm_ch = [merge_triangles(im_tris, mean_tris, i.astype(np.float32), (dissolve_frac)) for i in im_ch]
        morph_lst.append(newIm_ch)
    return morph_lst

In [31]:
im_lst = [skio.imread(imname[:-3] + 'jpg')/255 for imname in asf_lst]
pts_lst = [[(im.shape[1] * coord[0], im.shape[0] * coord[1]) for coord in coords] for im, coords in zip(im_lst, coord_lst) ]
first_im = im_lst[0]
coord_mean_scaled = [(first_im.shape[1] * coord[0], first_im.shape[0] * coord[1]) for coord in coord_mean]

In [32]:

plt.imshow(im_lst[1])
plt.show()

In [33]:
morph_lst = make_morph_lst(im_lst, coord_lst, coord_mean)

In [34]:
# print(morph_lst)


def get_channel(morph_lst, ch_num):
    ch = [m[ch_num] for m in morph_lst]
    line = np.zeros(ch[0].shape)
    for r in ch:
        line += r/(255 * 40)
    new_ch = np.dstack((line, line,line))
    return line



In [35]:
new_im = np.dstack(tuple([get_channel(morph_lst, i) for i in range(0,3)]))
print(new_im.shape)
plt.imshow(new_im)
plt.show()
fname = 'danish_avg.jpg'
skio.imsave(fname, new_im)

(480, 640, 3)


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/skimage/io/_io.py:132: UserWarning: danish_avg.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


In [36]:
# get the avg shape and show other danish faces morphing into avg shape
# show my face morph into avg shape
# show avg image morph into my face. 

In [37]:
# To show my face morph to avg shape, or any face morph to another shape, just plug in the points and set the fraction 
# always at 1 on my face, or write my own function

In [38]:
def morph_to_shape(im_pts, mean_pts, im, warp_frac):
    tri = Delaunay(mean_pts)
    tri_simp = tri.simplices
    im1_tris = np.array(im_pts)[tri_simp]
    im1_ch = np.dsplit(im, im.shape[-1])
    warp_pts = avg_shape(im_pts,mean_pts, warp_frac)
    warp_tris = warp_pts[tri_simp]
#     merging 
    newIm1_ch = [merge_triangles(im1_tris, warp_tris, i.astype(np.float32), 1) for i in im1_ch]
#     print('wuuuuuuuuut')
#     print(newIm1_ch)
    im1_n = np.dstack(tuple(newIm1_ch))
    return im1_n

In [39]:
def morph_to_shape_animation(imname, im_pts, mean_pts, im, frames, to_size=False):
#     we also need to multiple im_pts and mean_pts
    if (to_size == False):
        print('yay')
        im_pts =   [(im.shape[1] * coord[0], im.shape[0] * coord[1]) for coord in im_pts]
        mean_pts = [(im.shape[1] * coord[0], im.shape[0] * coord[1]) for coord in mean_pts]
        im_pts   += [(0.0,0.0), (float(len(im[0])),0), (float(len(im[0])), float(len(im))),(0.0, float(len(im)))]
        mean_pts += [(0.0,0.0), (float(len(im[0])),0), (float(len(im[0])), float(len(im))),(0.0, float(len(im)))]
    if (frames < 2):
        return "error, need more than 1 frames"
    increment = 1/(frames-1)
    for i in range(0, frames):
        new_im = morph_to_shape(im_pts, mean_pts, im, increment * i)
        fname = imname[:-4] + '_' + 'avg' + '_' + str(1000 + i) + '.jpg'
        skio.imsave(fname, np.clip(new_im,0.0,1.0))
    all_names = imname[:-4] + '_' + 'avg' + '_*' + '.jpg'
    o_name = imname[:-4] + '_' + 'avg' + '_merge.gif'
    os.system('convert -delay 10 {0} {1}'.format(all_names, o_name))

In [44]:
im = im_lst[0]
plt.imshow(im)
im_pts =   [(im.shape[1] * coord[0], im.shape[0] * coord[1]) for coord in coord_mean]
mean_pts = [(im.shape[1] * coord[0], im.shape[0] * coord[1]) for coord in coord_mean]
# im5 = morph_to_shape(coord_array[0], coord_mean, im, 1)

In [45]:
morph_to_shape_animation(asf_lst[0], coord_array[0], coord_mean, im_lst[0], 45)

yay


In [46]:
morph_to_shape_animation(asf_lst[25], coord_array[25], coord_mean, im_lst[25], 45)

yay


In [ ]:
# im_avg = skio.imread('danish_avg_small.jpg')/255
# im_avg = sp.misc.imresize(im_avg, (750,602,3))/255
# # low sf
# im_eric = skio.imread('eric_small.jpg')/255
# #   reshape
# im_eric = sp.misc.imresize(im_eric,(750,602,3) )/255
# pts_e,pts_f = get_points(im_eric, im_avg, 36)
# #     add corners
# pts_e += [(0.0,0.0), (float(len(im_avg[0])),0), (float(len(im_avg[0])), float(len(im_avg))),(0.0, float(len(im_avg)))]
# pts_f += [(0.0,0.0), (float(len(im_avg[0])),0), (float(len(im_avg[0])), float(len(im_avg))),(0.0, float(len(im_avg)))]

In [50]:
# caricature, just change to be negative. 
np.shape(im_avg)
plt.imshow(im_avg)
plt.show()
morph_to_shape_animation('eric_small.jpg', np.array(pts_e), np.array(pts_f), im_eric, 45, True)

KeyboardInterrupt: 

In [ ]:
morph_to_shape_animation('danish_avg_small.jpg', np.array(pts_f), np.array(pts_e), im_avg, 45, True)

In [ ]:
create_morph('eric_small.jpg', 'danish_avg_small.jpg', pts_e, pts_f, caricature=0.5)


In [ ]:
# ############################## Toggle coord points for extra credit, founding fathers
founding_fathers = [[(406.0194805194804, 128.15259740259728),
  (229.47727272727258, 126.12337662337654),
  (148.30844155844142, 306.72402597402584),
  (357.3181818181817, 343.24999999999989),
  (434.42857142857127, 345.27922077922068),
  (426.31168831168821, 418.33116883116872),
  (361.37662337662329, 471.09090909090901),
  (424.28246753246736, 471.09090909090901),
  (401.96103896103881, 560.37662337662323),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(365.43506493506476, 134.2402597402596),
  (243.68181818181807, 130.18181818181802),
  (136.13311688311671, 290.4902597402596),
  (314.70454545454533, 318.89935064935059),
  (395.87337662337649, 316.87012987012974),
  (375.58116883116867, 391.95129870129858),
  (326.87987012986997, 446.74025974025966),
  (389.78571428571416, 446.74025974025966),
  (365.43506493506476, 517.76298701298697),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(371.5227272727272, 154.53246753246742),
  (268.03246753246736, 140.32792207792193),
  (91.490259740259603, 329.04545454545439),
  (302.52922077922068, 365.5714285714285),
  (399.93181818181807, 371.65909090909082),
  (377.61038961038952, 438.62337662337654),
  (324.85064935064923, 497.47077922077915),
  (381.66883116883099, 503.55844155844147),
  (353.25974025974011, 572.55194805194799),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(353.25974025974011, 130.18181818181802),
  (201.0681818181817, 144.38636363636351),
  (140.1915584415583, 312.81168831168816),
  (316.73376623376606, 324.98701298701292),
  (401.96103896103881, 327.01623376623365),
  (397.90259740259722, 387.89285714285705),
  (334.99675324675314, 454.85714285714278),
  (406.0194805194804, 454.85714285714278),
  (389.78571428571416, 513.70454545454538),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(355.28896103896096, 126.12337662337654),
  (188.89285714285694, 142.35714285714278),
  (128.0162337662336, 302.66558441558436),
  (300.49999999999983, 312.81168831168816),
  (393.84415584415575, 300.63636363636351),
  (371.5227272727272, 373.68831168831161),
  (316.73376623376606, 454.85714285714278),
  (387.75649350649343, 452.82792207792198),
  (361.37662337662329, 519.79220779220771),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(355.28896103896096, 174.82467532467524),
  (213.24350649350635, 166.70779220779207),
  (140.1915584415583, 306.72402597402584),
  (314.70454545454533, 312.81168831168816),
  (389.78571428571416, 316.87012987012974),
  (381.66883116883099, 389.92207792207785),
  (328.90909090909082, 432.53571428571422),
  (393.84415584415575, 436.59415584415575),
  (369.49350649350635, 505.58766233766227),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(371.5227272727272, 118.00649350649337),
  (192.95129870129853, 142.35714285714278),
  (128.0162337662336, 314.84090909090901),
  (302.52922077922068, 375.71753246753235),
  (399.93181818181807, 357.45454545454538),
  (379.63961038961025, 448.76948051948045),
  (324.85064935064923, 499.49999999999994),
  (408.04870129870113, 495.44155844155836),
  (373.55194805194793, 568.4935064935064),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(375.58116883116867, 144.38636363636351),
  (245.71103896103881, 138.29870129870119),
  (166.57142857142838, 337.16233766233756),
  (278.17857142857127, 365.5714285714285),
  (371.5227272727272, 373.68831168831161),
  (334.99675324675314, 452.82792207792198),
  (280.20779220779201, 505.58766233766227),
  (353.25974025974011, 517.76298701298697),
  (318.76298701298691, 578.63961038961031),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(363.40584415584402, 101.77272727272714),
  (201.0681818181817, 120.03571428571411),
  (125.9870129870128, 329.04545454545439),
  (288.32467532467518, 333.10389610389598),
  (381.66883116883099, 329.04545454545439),
  (357.3181818181817, 404.12662337662329),
  (310.64610389610374, 456.88636363636357),
  (379.63961038961025, 458.91558441558431),
  (341.08441558441547, 533.9967532467532),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(359.34740259740244, 170.76623376623365),
  (190.92207792207779, 189.02922077922062),
  (144.24999999999983, 361.51298701298691),
  (304.55844155844142, 387.89285714285705),
  (393.84415584415575, 387.89285714285705),
  (375.58116883116867, 475.14935064935059),
  (324.85064935064923, 527.90909090909076),
  (395.87337662337649, 521.82142857142844),
  (363.40584415584402, 609.07792207792204),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(351.23051948051938, 113.94805194805178),
  (225.41883116883099, 130.18181818181802),
  (162.51298701298691, 306.72402597402584),
  (292.38311688311677, 308.75324675324669),
  (375.58116883116867, 306.72402597402584),
  (357.3181818181817, 371.65909090909082),
  (304.55844155844142, 432.53571428571422),
  (387.75649350649343, 420.36038961038952),
  (363.40584415584402, 493.41233766233756),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)],
 [(373.55194805194793, 132.21103896103887),
  (245.71103896103881, 138.29870129870119),
  (170.62987012986997, 290.4902597402596),
  (306.58766233766215, 290.4902597402596),
  (393.84415584415575, 294.54870129870119),
  (367.46428571428561, 357.45454545454538),
  (318.76298701298691, 410.21428571428561),
  (385.72727272727258, 414.2727272727272),
  (363.40584415584402, 489.35389610389603),
  (0.0, 0.0),
  (602.0, 0),
  (602.0, 750.0),
  (0.0, 750.0)]]

In [ ]:
# extra credit, let's do all of the founding fathers
founding_lst = [
    'thomas_jefferson.jpg',
    'george_washington.jpg',
    'john_adams.jpg',
    'alexander_hamilton.jpg',
    'james_madison.jpg',
    'john_jay.jpg',
    'patrick_henry.jpg',
    'john_marshall.jpg',
    'george_mason.jpg',
    'james_monroe.jpg',
    'sam_adams.jpg',
    'benjamin_franklin.jpg'
               ]
f_seq_1 = np.array(list(range(0,12)))
f_seq_2 = np.roll(f_seq_1, -1)
founding_points = []
# to get points yourself
# for i in range(0,6):
#     p1, p2 = pts_from_name(founding_lst[2*i], founding_lst[2*i+1],num_points = 9)
#     founding_points.append(p1)
#     founding_points.append(p2)

In [ ]:
founding_points_2 = [(create_morph(founding_lst[i],
                                   founding_lst[j],
                                   founding_points[i],
                                   founding_points[j])) for i,j in zip(f_seq_1,f_seq_2)]

In [ ]:
all_names = 'fathers_merge/*.jpg'
o_name = 'gifs/fathers.gif'

from os import listdir
from os.path import isfile, join
mypath='/Users/ericlinxie/cs194/otherProjects/proj4/fathers_merge'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [ ]:
for i,ofile in enumerate(onlyfiles):
    os.system('mv {0} {1}.jpg'.format(ofile,str(10000000000+i)))

os.system('convert -delay 10 {0} {1}'.format(all_names, o_name))